In [ ]:
from ib_insync import *
import pandas as pd
import numpy as np
import datetime

# IBKR Connection Settings
HOST = "127.0.0.1"
PORT = 7497  # 4001 for live trading, 4002 for paper trading
CLIENT_ID = 1  # Unique client ID

# Trading Settings
SYMBOL = "AAPL"
INITIAL_CAPITAL = 100  # Increased for testing real trading
SHORT_SMA = 10
LONG_SMA = 50
VOLATILITY_LOOKBACK = 20
RISK_TOLERANCE = 0.02
MARGIN_BUFFER = 0.9

# Market Hours (Netherlands, CET)
MARKET_OPEN = datetime.time(15, 30)
MARKET_CLOSE = datetime.time(22, 00)

async def connect_ibkr():
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=1)
    return ib

# This is how you should run async functions inside Jupyter
ib = await connect_ibkr()

# Define Stock Contract
contract = Stock(SYMBOL, "SMART", "USD", primaryExchange="NASDAQ")
ib.qualifyContracts(contract)

# Initialize Capital & Positions
capital = INITIAL_CAPITAL
positions = {SYMBOL: 0}
trade_log = []

# Function to Handle New Data
def on_bar_update(bars, has_new_bar):
    """ Processes live bar updates and executes trades when signals occur. """
    global capital, positions

    if not has_new_bar:
        return

    df = util.df(bars)
    df["SMA_Short"] = df["close"].rolling(SHORT_SMA).mean()
    df["SMA_Long"] = df["close"].rolling(LONG_SMA).mean()
    df["Volatility"] = df["close"].pct_change().rolling(VOLATILITY_LOOKBACK).std()
    df["Z_Score"] = (df["close"] - df["close"].rolling(VOLATILITY_LOOKBACK).mean()) / df["Volatility"]
    df.dropna(inplace=True)

    if len(df) < LONG_SMA:
        return  # Not enough data

    signal = check_trade_signal(df)
    if signal:
        execute_trade(signal, df.iloc[-1]["close"])

def check_trade_signal(df):
    """ Determines if a buy or sell signal is triggered. """
    if df.iloc[-2]["SMA_Short"] < df.iloc[-2]["SMA_Long"] and df.iloc[-1]["SMA_Short"] > df.iloc[-1]["SMA_Long"] and df.iloc[-1]["Z_Score"] < -1:
        return "BUY"
    elif df.iloc[-2]["SMA_Short"] > df.iloc[-2]["SMA_Long"] and df.iloc[-1]["SMA_Short"] < df.iloc[-1]["SMA_Long"] and df.iloc[-1]["Z_Score"] > 1:
        return "SELL"
    return None

def risk_based_position_sizing(price, volatility):
    """ Calculates position size based on risk tolerance and available capital. """
    if volatility > 0:
        risk_per_share = volatility * price
        max_shares = int((capital * MARGIN_BUFFER * RISK_TOLERANCE) / risk_per_share)
        return max_shares if max_shares > 0 else 1
    return 1

def execute_trade(order_type, price):
    """ Executes market order based on signal and updates capital. """
    global capital, positions

    size = risk_based_position_sizing(price, np.std(positions.values()))
    if size <= 0 or capital < price * size:
        print(f"{datetime.datetime.now()}: Not enough capital for {order_type} order.")
        return

    order = MarketOrder(order_type, size)
    trade = ib.placeOrder(contract, order)

    if order_type == "BUY":
        capital -= size * price
        positions[SYMBOL] += size
    elif order_type == "SELL":
        capital += size * price
        positions[SYMBOL] -= size

    trade_log.append({"timestamp": datetime.datetime.now(), "order": order_type, "size": size, "price": price})
    print(f"{datetime.datetime.now()}: Executed {order_type} order for {size} shares at ${price:.2f}")

# Subscribe to Real-Time Data
bars = ib.reqHistoricalData(
    contract,
    endDateTime="",
    durationStr="1 D",
    barSizeSetting="1 min",
    whatToShow="MIDPOINT",
    useRTH=True,
    formatDate=1,
    keepUpToDate=True
)

bars.updateEvent += on_bar_update

# Start IBKR Event Loop
print("📡 Running live trading with real-time updates...")
ib.run()

RuntimeError: This event loop is already running

Error 1300, reqId -1: Socket port has been reset and this connection is being dropped. Please reconnect on the new port -4001
Error 1300, reqId -1: Socket port has been reset and this connection is being dropped. Please reconnect on the new port -4001
Peer closed connection.


: 